In [1]:
import numpy as np
import pandas as pd
import copy as cp

In [2]:
def sigmoid(n):
    return (1/(1+np.exp(-n)))
def softmax(n):
    n=n-max(n)
    n=np.exp(n)
    return n/np.sum(n)

In [30]:
class hidden_layer():
    def __init__(self,x,N):
        temp=np.ones([x.shape[1]])
        temp.resize(1,x.shape[1])
        temp=np.append(temp,x,axis=0)
        self.x=temp
        self.N=N
        self.w=np.random.rand(self.x.shape[0],self.N)
        self.z=np.dot(np.transpose(self.w),self.x)
        self.a=sigmoid(self.z)
    def out(self):
        return self.a
    def back(self,delta):
        self.delta=delta*self.a*(1-self.a)
        self.dw=np.dot(self.x,np.transpose(self.delta))
        self.w=self.w-0.001*(self.dw)
        self.deltaw=np.dot(self.w[1:self.x.shape[0],:],self.delta)
        return self.deltaw
    def forward(self):
        self.z=np.dot(np.transpose(self.w),self.x)
        self.a=sigmoid(self.z)
    def forward_(self,x):
        temp=np.ones([x.shape[1]])
        temp.resize(1,x.shape[1])
        x=np.append(temp,x,axis=0)
        self.x=x
        self.z=np.dot(np.transpose(self.w),self.x)
        self.a=sigmoid(self.z)
        return self.a

In [31]:
class final_layer():
    def __init__(self,x,N):
        temp=np.ones([x.shape[1]])
        temp.resize(1,x.shape[1])
        temp=np.append(temp,x,axis=0)
        self.x=temp
        self.N=N
        self.w=np.random.rand(self.x.shape[0],self.N)
        self.z=np.dot(np.transpose(self.w),self.x)
        self.a=cp.deepcopy(self.z)
        self.a=sigmoid(self.z)
    def back(self,y):
        print("cost = ",end='')
        print(np.sum(np.square(self.a-y)))
        self.delta=(self.a-y)*self.a*(1-self.a)
        self.deltaw=np.dot(self.w[1:self.x.shape[0],:],self.delta)
        self.dw=np.dot(self.delta,np.transpose(self.x))
        self.w=self.w-0.001*np.transpose(self.dw)
        return self.deltaw
    def forward_(self,x):
        temp=np.ones([x.shape[1]])
        temp.resize(1,x.shape[1])
        temp=np.append(temp,x,axis=0)
        self.x=temp
        self.z=np.dot(np.transpose(self.w),self.x)
        self.a=sigmoid(self.z)
    def out(self):
        return self.a

In [52]:
class neural_network():
    def __init__(self,x,y):
        self.num_hid_layer=int(input("Enter Number of hidden Layers"))
        self.neurons_per_layer=list();
        for i in range (self.num_hid_layer):
            print("Enter number of neurons in layer ",+i+1)
            self.neurons_per_layer.append(int(input()))
        self.layer_list=[]
        self.x=x
        self.y=y
        self.l_rate=float(input("Enter learning rate"))
        self.epoch=int(input("Enter number of epochs"))
        temp=x
        for i in range (len(self.neurons_per_layer)):
            self.layer_list.append(hidden_layer(temp,self.neurons_per_layer[i]))
            temp=self.layer_list[i].out()
        self.layer_list.append(final_layer(temp,y.shape[0]))
    def train(self):
        for i in range (self.epoch):
            print("at epoch : ",end='')
            print(i,end=' ')
            for j in reversed(range (len(self.layer_list))):
                if (j==len(self.layer_list)-1):
                    temp=self.layer_list[j].back(self.y)
                else:
                    temp=self.layer_list[j].back(temp)
            for k in range (len(self.layer_list)):
                if (k==0):
                    self.layer_list[k].forward()
                    temp=self.layer_list[k].out()
                else:
                    self.layer_list[k].forward_(temp)
                    temp=self.layer_list[k].out()
        return temp.round()
    def test(self,x):
        temp=x
        for k in range (len(self.layer_list)):
            if (k==0):
                self.layer_list[k].forward_(temp)
                temp=self.layer_list[k].out()
            else:
                self.layer_list[k].forward_(temp)
                temp=self.layer_list[k].out()
        return temp

In [53]:
file=pd.read_csv("log.csv")
x=np.array(file['x'])
x.resize(1,len(x))
x=np.append(x,np.array([file['y']]),axis=0)
yt=np.array(file['yclass'])
y=np.array([])
if (yt[0]==0):
    y=np.append(y,np.array([1,0]),axis=0)
else:
     y=np.append(y,np.array([1,0]),axis=0)
y.resize([2,1])

for i in range(1,len(yt)):
    if (yt[i]==0):
        y=np.append(y,np.array([[1],[0]]),axis=1)
    else :
        y=np.append(y,np.array([[0],[1]]),axis=1)
    

In [55]:
#x and y is the training data
NN= neural_network(x,y)

Enter Number of hidden Layers2
Enter number of neurons in layer  1
32
Enter number of neurons in layer  2
16
Enter learning rate1
Enter number of epochs1000


In [56]:
res=NN.train()

at epoch : 0 cost = 719.5469587488278
at epoch : 1 cost = 719.5458060622937
at epoch : 2 cost = 719.5446467691557
at epoch : 3 cost = 719.5434808104714
at epoch : 4 cost = 719.5423081265875
at epoch : 5 cost = 719.5411286571303
at epoch : 6 cost = 719.539942340994
at epoch : 7 cost = 719.5387491163295
at epoch : 8 cost = 719.5375489205335
at epoch : 9 cost = 719.5363416902364
at epoch : 10 cost = 719.5351273612907
at epoch : 11 cost = 719.5339058687598
at epoch : 12 cost = 719.5326771469039
at epoch : 13 cost = 719.5314411291695
at epoch : 14 cost = 719.5301977481759
at epoch : 15 cost = 719.5289469357016
at epoch : 16 cost = 719.5276886226727
at epoch : 17 cost = 719.5264227391481
at epoch : 18 cost = 719.5251492143059
at epoch : 19 cost = 719.5238679764309
at epoch : 20 cost = 719.5225789528988
at epoch : 21 cost = 719.521282070162
at epoch : 22 cost = 719.5199772537358
at epoch : 23 cost = 719.5186644281827
at epoch : 24 cost = 719.5173435170964
at epoch : 25 cost = 719.516014443087

at epoch : 245 cost = 718.5419639617705
at epoch : 246 cost = 718.5267551978274
at epoch : 247 cost = 718.5112040301537
at epoch : 248 cost = 718.4952987313906
at epoch : 249 cost = 718.4790270339304
at epoch : 250 cost = 718.4623760985332
at epoch : 251 cost = 718.4453324807355
at epoch : 252 cost = 718.4278820948657
at epoch : 253 cost = 718.4100101754711
at epoch : 254 cost = 718.39170123593
at epoch : 255 cost = 718.3729390240139
at epoch : 256 cost = 718.3537064741296
at epoch : 257 cost = 718.3339856559512
at epoch : 258 cost = 718.3137577191201
at epoch : 259 cost = 718.293002833658
at epoch : 260 cost = 718.2717001257008
at epoch : 261 cost = 718.2498276081207
at epoch : 262 cost = 718.227362105556
at epoch : 263 cost = 718.2042791733181
at epoch : 264 cost = 718.1805530095851
at epoch : 265 cost = 718.1561563602252
at epoch : 266 cost = 718.1310604155217
at epoch : 267 cost = 718.1052346979872
at epoch : 268 cost = 718.0786469403531
at epoch : 269 cost = 718.0512629527288
at e

at epoch : 452 cost = 534.2168535472047
at epoch : 453 cost = 534.1195894787229
at epoch : 454 cost = 534.0198819098418
at epoch : 455 cost = 533.9176416029827
at epoch : 456 cost = 533.812778206962
at epoch : 457 cost = 533.7052005173149
at epoch : 458 cost = 533.5948167435636
at epoch : 459 cost = 533.481534931952
at epoch : 460 cost = 533.3652637740017
at epoch : 461 cost = 533.2459140740282
at epoch : 462 cost = 533.1234011094565
at epoch : 463 cost = 532.9976479641778
at epoch : 464 cost = 532.8685896490226
at epoch : 465 cost = 532.7361775018985
at epoch : 466 cost = 532.6003831002452
at epoch : 467 cost = 532.4612008618712
at epoch : 468 cost = 532.3186487394509
at epoch : 469 cost = 532.172766856529
at epoch : 470 cost = 532.0236143461055
at epoch : 471 cost = 531.8712647671448
at epoch : 472 cost = 531.7158002377107
at epoch : 473 cost = 531.5573041294489
at epoch : 474 cost = 531.395852270184
at epoch : 475 cost = 531.2315032879662
at epoch : 476 cost = 531.0642896417453
at e

at epoch : 657 cost = 425.9524500758884
at epoch : 658 cost = 425.308892286847
at epoch : 659 cost = 424.6671939142472
at epoch : 660 cost = 424.028021490692
at epoch : 661 cost = 423.39199959434995
at epoch : 662 cost = 422.759677594068
at epoch : 663 cost = 422.1315119024132
at epoch : 664 cost = 421.50786167608396
at epoch : 665 cost = 420.8889946815767
at epoch : 666 cost = 420.27509984533884
at epoch : 667 cost = 419.66630336155526
at epoch : 668 cost = 419.06268583908434
at epoch : 669 cost = 418.4642986755265
at epoch : 670 cost = 417.87117853784355
at epoch : 671 cost = 417.2833593545688
at epoch : 672 cost = 416.7008813870194
at epoch : 673 cost = 416.1237966208521
at epoch : 674 cost = 415.5521690818431
at epoch : 675 cost = 414.986068359357
at epoch : 676 cost = 414.4255554497891
at epoch : 677 cost = 413.87066231038165
at epoch : 678 cost = 413.3213693281119
at epoch : 679 cost = 412.7775864414515
at epoch : 680 cost = 412.2391426252244
at epoch : 681 cost = 411.70578534935

at epoch : 868 cost = 59.78736302653838
at epoch : 869 cost = 59.663719546585014
at epoch : 870 cost = 59.54200424202055
at epoch : 871 cost = 59.4221581187482
at epoch : 872 cost = 59.30412370478322
at epoch : 873 cost = 59.18784506379744
at epoch : 874 cost = 59.07326781620981
at epoch : 875 cost = 58.960339168105556
at epoch : 876 cost = 58.849007948097096
at epoch : 877 cost = 58.739224652028255
at epoch : 878 cost = 58.6309414951681
at epoch : 879 cost = 58.52411247124261
at epoch : 880 cost = 58.4186934173168
at epoch : 881 cost = 58.31464208317173
at epoch : 882 cost = 58.21191820343353
at epoch : 883 cost = 58.110483570319076
at epoch : 884 cost = 58.010302104484005
at epoch : 885 cost = 57.911339921117346
at epoch : 886 cost = 57.813565388148135
at epoch : 887 cost = 57.71694917323937
at epoch : 888 cost = 57.621464276171515
at epoch : 889 cost = 57.52708604328457
at epoch : 890 cost = 57.43379216087619
at epoch : 891 cost = 57.341562624853594
at epoch : 892 cost = 57.25037968

In [58]:
ress=NN.test(x).round()

In [27]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [59]:
accuracy_score(np.transpose(y),np.transpose(ress))

0.9555555555555556